In [49]:
import os
import sys
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2
from toolsets.file_io import get_file_list, readin_peak_list
from toolsets.feature_alignment import initilize_pmz_rt_list, find_feature, find_istd, clean_bad_features
from tqdm import tqdm
# from toolsets.feature_alignment import
from toolsets.search import quick_search_values, string_search
# from feature_finding.feature_finding_alphapept import find_feature

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
working_dir ='/Users/fanzhou.kong/Dropbox (Brightseed)/Mac/Documents/GitHub/data_garage/brighseed/processing'
istd_info = pd.read_csv('/Users/fanzhou.kong/Dropbox (Brightseed)/Mac/Documents/GitHub/data_garage/brighseed/istd_info.csv')

In [3]:
file_list = get_file_list(working_dir, '.csv')

In [72]:
lst =[1,2,3]

In [73]:
lst[0]=5

In [74]:
lst

[5, 2, 3]

In [4]:
qc_list = []
for file in file_list:
    if 'QC' in file:
        qc_list.append(file)
# qc_list[1:]
qc_paths = [os.path.join(working_dir, x+'.csv') for x in qc_list]

In [5]:
pmz_list, rt_list = initilize_pmz_rt_list(qc_paths)
print(len(pmz_list))

0


100%|██████████| 13060/13060 [00:18<00:00, 689.32it/s]

27875


In [6]:
file_paths = [os.path.join(working_dir, x+'.csv') for x in file_list]
alignment_result= pd.DataFrame(zip(pmz_list, rt_list), columns=['pmz', 'rt'])
for file in tqdm(file_paths):
    file_temp = pd.read_csv(file)
    intensity_temp = []
    # pmz_temp = []
    for i in (range(0, len(pmz_list))):
        feature_temp = find_feature(file_temp, mz = pmz_list[i], rt =rt_list[i])
        if len(feature_temp)>0:
            intensity_temp.append(feature_temp['Height'].sum())
            # pmz_temp.append()
            # break
        else:
            intensity_temp.append(0)
    alignment_result[os.path.basename(file).split('.')[0]]=intensity_temp
    # break

100%|██████████| 41/41 [50:36<00:00, 74.07s/it]


In [79]:
file_paths = [os.path.join(working_dir, x+'.csv') for x in file_list]
alignment_result= pd.DataFrame(zip(pmz_list, rt_list), columns=['pmz', 'rt'])
tt = pd.DataFrame()
for file in tqdm(file_paths[0:1]):
    file_temp = pd.read_csv(file)
    intensity_temp = []
    feature_temp = find_feature(file_temp, mz = 488.1655, rt = 5.73458 )
    tt = pd.concat([tt, feature_temp])
    # pmz_temp = []

    # alignment_result[os.path.basename(file).split('.')[0]]=intensity_temp

100%|██████████| 41/41 [00:02<00:00, 20.17it/s]


In [104]:
for i in file_paths:
    if '32' in i:
        break

In [105]:
i

'/Users/fanzhou.kong/Dropbox (Brightseed)/Mac/Documents/GitHub/data_garage/brighseed/processing/1870_AX_M5P2_M7P1_M7_X32_P.csv'

In [106]:
file_temp = pd.read_csv(i)

In [107]:
find_feature(file_temp, mz = 201.12767, rt = 3.401)

,PeakID,Title,Scans,RT left(min),RT_adjusted,RT (min),RT right (min),Precursor m/z,Height,Area,...,m/z matched,MS/MS matched,RT similarity,Dot product,Reverse dot product,Fragment presence %,Total score,S/N,MS1 isotopes,MSMS spectrum
2260,4672,Unknown,1006,3.221743,3.410807,3.419923,3.648793,201.1274,5108583.5,36057804.0,...,False,False,NaN,NaN,NaN,NaN,NaN,99128.73,201.09109:1550612 201.11212:133751 201.12741:6...,NaN


In [108]:
alignment_result

,pmz,rt
0,128.3630,0.236472
1,253.4384,0.478195
2,100.9567,0.548646
3,119.9751,0.548646
4,123.0037,0.548646
...,...,...
27870,1426.8900,3.167323
27871,1438.6430,9.025468
27872,1439.3000,5.313295
27873,1452.5210,4.434927


In [98]:
file_temp = pd.read_csv(qc_paths[1])

In [99]:
feature_temp = find_feature(file_temp, mz = 488.1655, rt = 5.73458 )

In [100]:
feature_temp

,PeakID,Title,Scans,RT left(min),RT_adjusted,RT (min),RT right (min),Precursor m/z,Height,Area,...,m/z matched,MS/MS matched,RT similarity,Dot product,Reverse dot product,Fragment presence %,Total score,S/N,MS1 isotopes,MSMS spectrum
8206,9194,Unknown,1630,5.534427,5.733026,5.739232,5.940308,488.1663,10930.682617,47386.007812,...,False,False,NaN,NaN,NaN,NaN,NaN,218.6137,488.16626:35339 490.28107:22265 493.11539:21780,NaN


In [51]:
from toolsets.file_io import readin_alignment

In [52]:
msdial_output = readin_alignment('/Users/fanzhou.kong/Dropbox (Brightseed)/Mac/Documents/GitHub/data_garage/msdial_alignment/Height_0_20237101534.txt')

In [81]:
quick_search_values(msdial_output, 'Average Mz', 488.1659-0.01, 488.1659+0.01, ifsorted=False)

,Alignment ID,Average Rt(min),Average Mz,Metabolite name,Adduct type,Post curation result,Fill %,MS/MS assigned,Reference RT,Reference m/z,...,1870_AX_M5P2_M7P1_M7_X56_P,1870_AX_M5P2_M7P1_M7_X57_P,1870_AX_M5P2_M7P1_M7_X58_P,1870_AX_M5P2_M7P1_M7_X59_P,QC_Pool1_P,QC_Pool2_P,QC_Pool3_P,1,1.1,peaks


In [61]:
quick_search_values(msdial_output, 'Average Mz', 465.180858-0.01, 465.180858+0.01, ifsorted=False)

,Alignment ID,Average Rt(min),Average Mz,Metabolite name,Adduct type,Post curation result,Fill %,MS/MS assigned,Reference RT,Reference m/z,...,1870_AX_M5P2_M7P1_M7_X56_P,1870_AX_M5P2_M7P1_M7_X57_P,1870_AX_M5P2_M7P1_M7_X58_P,1870_AX_M5P2_M7P1_M7_X59_P,QC_Pool1_P,QC_Pool2_P,QC_Pool3_P,1,1.1,peaks
19558,19558,3.820,465.17303,Unknown,[M+H]+,NaN,0.051,False,NaN,NaN,...,0,0,0,0,88832,102014,98610,39433.137032,73026.550629,NaN
19559,19559,4.093,465.17340,Unknown,[M+H]+,NaN,0.051,False,NaN,NaN,...,152009,201337,81893,17732,78208,71028,79120,33346.436004,55113.150632,NaN
19560,19560,3.968,465.17355,Unknown,[M+H]+,NaN,0.026,False,NaN,NaN,...,25389,7720,0,0,78866,84070,70676,41763.890274,79979.172150,NaN
19561,19561,3.884,465.17371,Unknown,[M+H]+,NaN,0.026,False,NaN,NaN,...,0,0,0,0,88832,102014,98610,40642.443990,77260.373810,NaN
19562,19562,4.019,465.17398,Unknown,[M+H]+,found in higher mz's MsMs_20354,0.051,False,NaN,NaN,...,89009,26695,5230,717,88321,84070,83973,40288.253362,78014.080839,NaN
19563,19563,4.208,465.17480,Unknown,[M+H]+,ion correlated with 17822; ion correlated with...,0.026,False,NaN,NaN,...,100112,442031,255623,69901,29049,49467,53615,27981.152463,82086.687868,NaN
19564,19564,5.736,465.18091,Unknown,[M+H]+,ion correlated with 20543,0.231,True,NaN,NaN,...,10730,9757,13242,12259,46355,54774,60727,61325.111579,81140.359991,126.23167\t7904\n173.45764\t11706\n285.92523\t...
19565,19565,4.250,465.18140,Unknown,[M+H]+,ion correlated with 17822; ion correlated with...,0.077,False,NaN,NaN,...,155117,442031,255623,69901,48418,49467,53615,28737.157792,83592.915801,NaN


In [62]:
file_temp = pd.read_csv(os.path.join(working_dir, '1870_AX_M5P2_M7P1_M7_X32_P.csv'))

In [64]:
quick_search_values(file_temp, 'Precursor m/z', 465.180858-0.01, 465.180858+0.01, ifsorted=False)

,PeakID,Title,Scans,RT left(min),RT_adjusted,RT (min),RT right (min),Precursor m/z,Height,Area,...,m/z matched,MS/MS matched,RT similarity,Dot product,Reverse dot product,Fragment presence %,Total score,S/N,MS1 isotopes,MSMS spectrum
8113,10612,Unknown,1658,5.612382,5.735056,5.733913,6.012498,465.1811,261420.390625,1.324556e+06,...,False,False,NaN,NaN,NaN,NaN,NaN,5228.4080,465.18112:588840 465.30103:21275 466.18402:124...,126.23167:7904 173.45764:11706 285.92523:9274 ...
8114,11275,Unknown,1759,5.942320,6.089834,6.081813,6.348814,465.1888,15600.044922,7.202996e+04,...,False,False,NaN,NaN,NaN,NaN,NaN,312.0009,465.18857:37997 467.28021:48023,NaN


In [7]:
alignment_result.sort_values(by = 'rt', ascending=True, inplace = True, ignore_index=True)
alignment_result = find_istd(alignment_result, istd_info)
alignment_result_refined = clean_bad_features(alignment_result)
alignment_result_refined.to_csv(os.path.join('/Users/fanzhou.kong/Dropbox (Brightseed)/Mac/Documents/GitHub/data_garage/brighseed/result', 'alignment_result2_refined.csv'), index = False)